### Flash ReRank

Models are better at using relevant information that occurs at the very begging (primacy bias) or end of it's input context (recency bias), and performance degrades significantly when models have to use information located in the middle of the input context.

- Cross Encoders
- Zero shot rerankers

In [1]:
%pip install -qqU flashrank

Note: you may need to restart the kernel to use updated packages.


In [2]:
from flashrank import Ranker

ranker = Ranker()

#### Small Ranker (~34MB), slightly slower & best performance (ranking precision)

In [3]:
ranker_small = Ranker(model_name="ms-marco-MiniLM-L-12-v2",
                      cache_dir="../.cache/")

#### Medium (~110MB), slower model with best zeroshot performance (ranking precision) 
on out of domain data.

In [4]:
ranker_medium_t5 = Ranker(model_name="rank-T5-flan", cache_dir="../.cache/")

# Medium (~150MB), slower model with competitive performance (ranking precision) 
for 100+ languages  (don't use for english)

In [5]:
ranker_medium_int = Ranker(
    model_name="ms-marco-MultiBERT-L-12", cache_dir="../.cache/")

- Metadata is optimal. 
- IDs come from retrieval DB or simple numeric indices

In [6]:
query = "How to speedup LLMs?"

In [7]:
passages = [
    {
        "id": 1,
        "text": "Introduce *lookahead decoding*: - a parallel decoding algo to accelerate LLM inference - w/o the need for a draft model or a data store - linearly decreases # decoding steps relative to log(FLOPs) used per decoding step.",
        "meta": {"additional": "info1"},
    },
    {
        "id": 2,
        "text": "LLM inference efficiency will be one of the most crucial topics for both industry and academia, simply because the more efficient you are, the more $$$ you will save. vllm project is a must-read for this direction, and now they have just released the paper",
        "meta": {"additional": "info2"},
    },
    {
        "id": 3,
        "text": "There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramatically reduces the model's memory footprint. - AutoGPTQ: Time-consuming but leads to a much smaller model and faster inference. The quantization is a one-time cost that pays off in the long run.",
        "meta": {"additional": "info3"},
    },
    {
        "id": 4,
        "text": "Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.",
        "meta": {"additional": "info4"},
    },
    {
        "id": 5,
        "text": "vLLM is a fast and easy-to-use library for LLM inference and serving. vLLM is fast with: State-of-the-art serving throughput Efficient management of attention key and value memory with PagedAttention Continuous batching of incoming requests Optimized CUDA kernels",
        "meta": {"additional": "info5"},
    },
]

In [8]:
from flashrank import RerankRequest

rerank_request = RerankRequest(query=query, passages=passages)
results = ranker_small.rerank(rerank_request)
results

[{'id': 4,
  'text': 'Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.',
  'meta': {'additional': 'info4'},
  'score': 0.95156515},
 {'id': 3,
  'text': "There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and drama

#### Sample #2 - What is the US Capital?

In [9]:
query = "What is the capital of the United States?"

In [23]:
passages = [
    {
        "id": 1,
        "text": "Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.",
        "meta": {},
    },
    {
        "id": 2,
        "text": "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
        "meta": {},
    },
    {
        "id": 3,
        "text": "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
        "meta": {},
    },
    {
        "id": 4,
        "text": "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. ",
        "meta": {},
    },
    {
        "id": 5,
        "text": "The city of Washington, D.C. is not only known for its significant role in the United States' government but also as the capital city. It houses the President's residence, the White House, and serves as the hub for all three branches of the federal government.",
        "meta": {},
    },
    {
        "id": 6,
        "text": "The economic capital of the United States is often considered to be New York City, due to its status as the home of the New York Stock Exchange and being a major hub for financial, cultural, and business activities.",
        "meta": {},
    },
    {
        "id": 7,
        "text": "In the heart of the nation, Washington, D.C. stands out as the capital, where the U.S. Capitol Building, a symbol of the country's democracy, is located. This city is central to American politics and history, hosting numerous national landmarks.",
        "meta": {},
    },
    {
        "id": 8,
        "text": "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
        "meta": {},
    },
    {
        "id": 9,
        "text": "North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck.",
        "meta": {},
    },
    {
        "id": 10,
        "text": "The debate over the capital punishment in the United States often takes center stage in Washington, D.C., where policymakers and activists gather to discuss its implications on justice and human rights.",
        "meta": {},
    },
    {
        "id": 11,
        "text": "Capital cities, including Washington, D.C., and state capitals like Olympia in Washington State, highlight the diversity of the United States. Each capital, with its unique history and culture, contributes to the rich tapestry that defines the American experience, from government operations to local heritage.",
        "meta": {},
    },
]

In [24]:
rerank_request = RerankRequest(query=query, passages=passages)
results = ranker_medium_t5.rerank(rerank_request)
print("=" * 10)
print("What???? HORRIBLE")
print("Medium T5 fails")
results

What???? HORRIBLE
Medium T5 fails


[{'id': 9,
  'text': 'North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck.',
  'meta': {},
  'score': 0.4610634},
 {'id': 2,
  'text': 'The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.',
  'meta': {},
  'score': 0.42852578},
 {'id': 1,
  'text': 'Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.',
  'meta': {},
  'score': 0.41662252},
 {'id': 5,
  'text': "The city of Washington, D.C. is not only known for its significant role in the United States' government but also as the capital city. It houses the President's residence, the White House, and serves as the hub for all three branches of the federal government.",
  'meta': {},
  'score': 0.39218792},
 {'id': 10,
  'text': 'Th

In [27]:
rerank_request = RerankRequest(query=query, passages=passages)
results = ranker_small.rerank(rerank_request)
print("=" * 10)
print("Hummmmm ... The NY passage ranks TOP ... Awful")
print("At least capital punishment is ranked last")
print("Capital punishment ranks #2?")
results

Hummmmm ... The NY passage ranks TOP ... Awful
At least capital punishment is ranked last
Capital punishment ranks #2?


[{'id': 6,
  'text': 'The economic capital of the United States is often considered to be New York City, due to its status as the home of the New York Stock Exchange and being a major hub for financial, cultural, and business activities.',
  'meta': {},
  'score': 0.9995332},
 {'id': 4,
  'text': 'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. ',
  'meta': {},
  'score': 0.9994824},
 {'id': 7,
  'text': "In the heart of the nation, Washington, D.C. stands out as the capital, where the U.S. Capitol Building, a symbol of the country's democracy, is located. This city is central to American politics and history, hosting numerous national landmarks.",
  'meta': {},
  'score': 0.9979709},
 {'id': 11,
  'text': 'Capital cities, including Washington, D.C., and state capitals like Olympia in Washington State, highlight the diversity of the United States. Each capital, with its

In [29]:
rerank_request = RerankRequest(query=query, passages=passages)
results = ranker_medium_int.rerank(rerank_request)
print("=" * 10)
print("Hummmmm ... Wasn't expecting much and the results are bad.")
results

Hummmmm ... Wasn't expecting much and the results are bad.


[{'id': 6,
  'text': 'The economic capital of the United States is often considered to be New York City, due to its status as the home of the New York Stock Exchange and being a major hub for financial, cultural, and business activities.',
  'meta': {},
  'score': 0.9698887},
 {'id': 11,
  'text': 'Capital cities, including Washington, D.C., and state capitals like Olympia in Washington State, highlight the diversity of the United States. Each capital, with its unique history and culture, contributes to the rich tapestry that defines the American experience, from government operations to local heritage.',
  'meta': {},
  'score': 0.9688047},
 {'id': 1,
  'text': 'Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.',
  'meta': {},
  'score': 0.96773326},
 {'id': 9,
  'text': 'North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of g